In [ ]:
from keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from scipy import spatial
from statistics import mode
import sklearn.metrics.pairwise
from statistics import mean
import random
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import spatial
from scipy.sparse import csr_matrix

In [ ]:
cate = ['alt.atheism','comp.graphics', 'misc.forsale','rec.autos','rec.sport.baseball','sci.crypt','sci.space','soc.religion.christian','talk.politics.guns','talk.politics.misc']

ng_train = fetch_20newsgroups(subset='train', categories = cate )
v = TfidfVectorizer()
train_y = ng_train.target
train_vector = v.fit_transform(ng_train.data)
sample_train = train_vector[:2000]
sample_train = csr_matrix.toarray(sample_train)

print(len(sample_train[0]))

64213


In [ ]:
#K means algorithm
k = 10
cluster_assignments = defaultdict(list)
cluster_idx = defaultdict(list)

#Initialization of clusters
cluster_index = random.sample(range(0,2000),k)
cluster_list = [sample_train[i] for i in cluster_index]

#Distance matrix
Euc_result = sklearn.metrics.pairwise.euclidean_distances(sample_train,cluster_list)
for i in range(len(sample_train)):
  cluster_num = np.argsort(Euc_result[i])[0]
  cluster_assignments[cluster_num].append(sample_train[i])


iter = 0
while(iter < 100):

#M Step
  new_assignments = defaultdict(list)
  new_cluster_list = np.array([])

  for i in range(k):
    new_cluster = np.average(cluster_assignments[i], axis = 0)
    new_cluster_list = np.append(new_cluster_list, new_cluster)
       
  new_cluster_list = np.reshape(new_cluster_list,(-1,64213))

  
  if np.array_equal(new_cluster_list,cluster_list):
    print('Entering if statement')
    break

  cluster_list = new_cluster_list
  cluster_assignments = defaultdict(list)
  cluster_idx = defaultdict(list)


#E step
  Euc_result = sklearn.metrics.pairwise.euclidean_distances(sample_train,cluster_list)
  for i in range(len(sample_train)):
    cluster_num = np.argsort(Euc_result[i])[0]
    cluster_assignments[cluster_num].append(sample_train[i])
    cluster_idx[cluster_num].append(i)
  iter +=1
#
print('num of iterations ', iter)
purity = []
counted = []
count = 0
for i in range(len(cluster_idx)):
  labels = []
  for x in cluster_idx[i]:
    label = train_y[x]
    labels.append(label)
  #print(len(labels))
  label_mode = mode(labels)
  purity.append(labels.count(label_mode))
purity_num = sum(purity)/len(sample_train)
print('Average Purity is ', purity_num)


# #Gini Index
# #Sum of prob picking cluster k * 1-prob picking cluster k
# #Calculate probabilities
all_nums = len(sample_train)
probs = []
for i in range(k):
  prob = len(cluster_idx[i])/all_nums
  probs.append(prob)
gini_index = 0
for i in probs:
  gini_index = gini_index + (i*(1-i))
print('gini index', gini_index)

Entering if statement
num of iterations  13
Average Purity is  0.3225
gini index 0.7578305
